# Tools-driven Mock Case Study of Type 2 Diabetes

In this module, we will:
1. Load and explore synthetic EHR data generated using Synthea.
2. Identify patients diagnosed with Type 2 Diabetes using SNOMED CT codes.
3. Analyze Hemoglobin A1C (HbA1c) levels in diabetic and non-diabetic patients.
4. Visualize data distributions and perform statistical testing.
5. Discuss findings and explore possible extensions.
6. Along the way, learn core python packages and tools including Pandas, Numpy, Scipy, Matplotlib, and Seaborn

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import scipy as sp
import os
from tqdm import tqdm

## I) Loading Relevant Data Into Pandas DataFrames, Stitching, and Inspecting

To load data into a Pandas Dataframe, you can use the `pandas.read_*` suite of functions. Common examples are.
- `pandas.read_csv()` to read from a comma separated values file
- `pandas.read_sql()` to read directly from a SQL database (either table by name or SQL query constructed as a string)
- `pandas.read_table()` to read from a more general delimited tabular file (could be comma, tab, space, `'|'`, or otherwise delimited)
- `pandas.read_json()` to read from a .json file, but pandas will expect the JSON data to have a particular structure
- `pandas.read_parquet()` to read from an Apache Parquet columnar store file

Below, we define a function that uses `pandas.read_parquet()` to load the Synthea data from parquet files

Data for 1,000 synthetic patients, divided into four states, was generated using the [Synthea tool](https://synthea.mitre.org/)
This data is split into four separate directories
```
output_hi/ # patients in Hawaii
output_ma/ # patients in Massacusetts
output_tx/ # patients in Texas
output_wa/ # patients in Washington
```

In [ ]:
# helper function for loading data from the Github repository
def load_data_from_github(filename):
    url = os.path.join('https://github.com/expmed/arch_workshop_core_python_skills_ws13/raw/refs/heads/main/data', filename)
    return pd.read_parquet(url) # uses pandas.read_parquet() to read the parquet() formatted data from the repository

### Skill Introduction - Concatenation of DataFrames
- If we want to work with a single dataset of all patients in a single DataFrame, we can use the built-in `pandas.concat()` function
- `pandas.concat()` can take in a bracketed list of dataframes, such as `[df1, df2, df3,...]` and concatenate them together row-wise into a single table
- The only requirement (or recommendation) is that all dataframes to be appended in this way have the same schema (names, types, and number of columns)
- Since the data for the four simulated states is split into separate files, we use `pd.concat()` to stitch together the data into individual DataFrames

In [ ]:
# first load in the patients files for the four states
patients_hi = load_data_from_github('output_hi_small/parquet/patients.parquet')
patients_ma = load_data_from_github('output_ma_small/parquet/patients.parquet')
patients_tx = load_data_from_github('output_tx_small/parquet/patients.parquet')
patients_wa = load_data_from_github('output_wa_small/parquet/patients.parquet')

# place all patients dataframes in a list
patients_dfs = [patients_hi, patients_ma, patients_tx, patients_wa]
# concatenate them
patients = pd.concat([patients_hi, patients_ma, patients_tx, patients_wa])

In [ ]:
# confirming that the consolidated patients dataframe has the same number of rows as all four dataframes
assert(len(patients) == sum([len(df) for df in patients_dfs]))
# confirming that the consolidated dataframe has the same columns as the individual ones
assert(len(patients.columns) == len(patients_hi.columns))

In [ ]:
# now define an additional helper function to simplify loading and concatenating data for the other files
def load_consolidated_data_for_file(filename):
    print(f"Loading data for {filename}")
    # uses a list comprehension to load the data for each of the four states into a list
    df = pd.concat([ # use pd.concat to append/concatenate the data for all states together into a single frame
        load_data_from_github(f"{output_dir}/parquet/{filename}") # use read_csv to load the data from each output directory
        for output_dir in tqdm(['output_hi_small', 'output_ma_small', 'output_tx_small', 'output_wa_small']) # loop over each output directory
    ])
    return df

In [ ]:
# load in the conditions, observations, and medications files
conditions = load_consolidated_data_for_file('conditions.parquet')
observations = load_consolidated_data_for_file('observations.parquet')
medications = load_consolidated_data_for_file('medications.parquet')

### Skill Introduction - Inspecting DataFrames
If we want to see a summary of the contents of a DataFrame, we can use the following methods and properties
1. We can use `df.head(n)` to display the first `n` rows of a DataFrame
2. We can access the `.columns` property of a DataFrame to display the names of all of the columns
3. We can access the `.dtypes` property of a DataFrame to show the columns and their datatypes

In [ ]:
# display the first 5 rows
conditions.head(5)

In [ ]:
# display the columns in the DataFrame
conditions.columns

In [ ]:
# display the columns and their datatypes
conditions.dtypes

## I.b Selecting Patients with Type 2 Diabetes

For this mock investigation, we are interested in patients who have been diagnosed with Type 2 Diabetes \
In order to extract the patients with Type 2 diabetes, we can use the `conditions` table, which stores \
[SNOMED CT](https://www.snomed.org/what-is-snomed-ct) coded diagnoses of various conditions, including Type 2 diabetes. \
While we can easily google/look up the SNOMED CT code for type 2 diabetes and use that to filter our `conditions` \
for instructive purposes, we will learn how to use Pandas search through the data to find which code we should use.

### Skill Introduction - Searching and Filtering Through DataFrames
- If we want to see the values that a particular column/`Series` in a DataFrame has, we can index into the dataframe with `df[<column_name>]`
- If we want to see all of the possible values that a column takes along with the number of records containing each value, we can use `df[<column_name>].value_counts()`
- If we want to search a text-based/`string`/`object` type column, we can use the `df[<column_name>].str.contains(<pattern>, ...)` method
- If we want to filter a DataFrame to only include specific values for a column, we can use boolean indexing such as `df[df[<column_name>] == value]`
- If we want to count the number of unique values present in a column, we can use `df[<column_name>].nunique()`
Below, we use all three of these to hone in on the patients with Type 2 Diabetes diagnoses

In [ ]:
# first we index into the DESCRIPTION column to see its contents
conditions['DESCRIPTION']

In [ ]:
# we use value counts to see a frequency breakdown of the various condition descriptions
conditions['DESCRIPTION'].value_counts().head(20) # we also use head(20) to show the top 20

In [ ]:
# now we use the .str.contains() method to find conditions with the substring "diabetes" in the description
diabetes_related_conditions = conditions[
    conditions['DESCRIPTION'].str.contains('diabetes', case=False) # uses case=False to make it case-insensitive search
]
diabetes_related_conditions['DESCRIPTION'].value_counts()

We notice from the above summary that there are patients diagnosed explicitly with `Diabetes mellitus type 2 (disorder)`, and there are also numerous patients diagnosed with co-morbidities described as being "due to" type 2 diabetes mellitus. In order to ensure that we capture all patients with Type 2 diabetes, we use additional substring filtering

In [ ]:
type2_diabetes = conditions[
    conditions['DESCRIPTION'].str.contains('type 2', case=False) & # Uses the '&' operation to take the conjunction of the two boolean indices
    conditions['DESCRIPTION'].str.contains('diabetes', case=False)
]

In [ ]:
type2_diabetes['DESCRIPTION'].value_counts()

In [ ]:
# next we count the unique patients with Type 2 diabetes
type2_diabetes['PATIENT'].nunique()

## I.c) Analysis of Hemoglobin A1c (HbA1c) Labs in Type 2 and Non-Type 2 patients
Now we will analyze Hemoglobin A1C levels for patients with and without a type 2 diabetes diagnosis \
To do this, we first filter the observations table for all hemoglobin A1C lab results \
We can use [LOINC](https://en.wikipedia.org/wiki/LOINC) code: `4548-4`

In [ ]:
hemoglobin_a1c = observations[observations['CODE'] == '4548-4']

In [ ]:
hemoglobin_a1c

### Skill Introduction - Assigning new Variables to a DataFrame
- To assign a new variable to a dataframe, we can call the `df.assign()` method which returns a new copy of the dataframe with the added variable(s)
- You can also assign directly to a newly named variable by indexing it, such as `df['variable'] = value(s)`, but this directly mutates the DataFrame
Next, we add another variable/column to the `hemoglobin_a1c` data to label patients with/without a Type2 diabetes diagnosis. \ Below, we demonstrate a DataFrame variable creation wherein we performing multiple operations at once

1. The outermost operation is a call to `hemoglobin_a1c.assign()` to assign a new variable/column to the dataframe of HbA1c labs
2. To the new variable `HASDIABETES`, we are assigning a series
3. The series we are assigning is a boolean series created by calling the `.isin()` method, effectively asking if the `PATIENT` ID of the record is in the set of IDs in the type 2 diabetes diagnoses
4. On this boolean series (which by default has values `True` and `False`) we are transforming the values into integers (1 for True, 0 for False) using `.astype('int')` to convert to integers

*NOTE:* We are cheating/simplifying somewhat. Really what we probably should do is label the HbA1c labs based on whether they occured before/after the diagnosis of Type 2 diabetes (within a certain time delta) to perhaps get a more accurate labeling of the labs. However, this would require creating an auxiliary data structure and lead to more complicated code, so we use this simpler approach for sake of brevity.

In [ ]:
hemoglobin_a1c_labeled = hemoglobin_a1c.assign( # calling .assign() to assign a new variable/column to the dataframe
    HASDIABETES=hemoglobin_a1c['PATIENT'].isin(type2_diabetes['PATIENT']).astype('int') # creating a new column named 'HASDIABETES'
)

In [ ]:
hemoglobin_a1c_labeled['HASDIABETES'].value_counts()

### Skill Introduction - Visualization of Data Distributions with Seaborn
The seaborn package can be used to visualize the distribution of continuous and categorical variables in data through a suite of built in plots
- `seaborn.kdeplot()` can be used to visualize a histogram with a curve fit to it via Kernel Density Estimation (KDE) to summarize a distribution
- `seaborn.violinplot()` can also be used to visualize the distribution of data, showing both density and summary stats like median and IQR
- We can also use the Pandas method `df[<column_name>].describe()` to produce summary stats for the data in a particular column

In [ ]:
# plot a Kernel Density Estimate of the distrubtion (probability density function) of Hemoglobin A1C lab measurements for Type 2 and Non Type 2 Patients
sns.kdeplot(
    hemoglobin_a1c_labeled.query('HASDIABETES == 1')['VALUE'].astype('float'), # Plotting the lab result VALUE for patients with Type 2
    label="With Diabetes", color="red", alpha=0.4, common_norm=False # plotting as a red curve with an appropriate label
)
sns.kdeplot(
    hemoglobin_a1c_labeled.query('HASDIABETES == 0')['VALUE'].astype('float'),  # Plotting the data for the patients without Type 2 diabetes
    label="Without Diabetes", color="blue", alpha=0.4, common_norm=False # plotting as a blue curve
)
# add a legend
plt.legend()
# change label of the xaxis
plt.xlabel('HbA1C (%)')
plt.show()

In [ ]:
# use seaborn to create a violin plot of the two distributions
sns.violinplot(
    hemoglobin_a1c_labeled.astype({'VALUE': float}),
    x='HASDIABETES',
    y='VALUE'
)
# change the label of the y-axis
plt.ylabel('HbA1C (%)')
plt.show()

In [ ]:
hemoglobin_a1c_labeled.astype({'VALUE': 'float'}).groupby('HASDIABETES')['VALUE'].describe()

The U.S. CDC has the following reference values published for Hemoglobin A1C

Normal: below 5.7% \
Prediabetes: 5.7% to 6.4% \
Diabetes: 6.5% or above 

In looking at the distribution plots above, we see that patients without a Type 2 diabetes diagnosis have an A1c distribution that is roubhly bi-modal, with one mode below 4.0% and another within the Pre-diabetes range of 5.7% to 6.4%

We also see that the patients with Type 2 diabetes diagnosis have a multi-modal HbA1c distribution, and a long tail that extends as high as to roughly 12%

### Skill Introduction - Statistical Testing with Scipy
If we want to quantify whether there is a significant difference in the two distributions \
one option is the use the Mann-Whitney U (Wilcoxon rank-sum) test \
provided by scipy's `mannwhitneyu()` function.

Another test we can perform is the Kolmogorov Smirnov test to test for subtle differences in shape of the two distributions.
We can perform a two-sample KS test using Scipy's `ks_2samp()` function.

The Mann-Whitney U test is mostly sensitive to changes in the median.
The KS test is sensitive to any differences in the two distributions (shape, spread, or median)

In [ ]:
U1, p = sp.stats.mannwhitneyu(
    hemoglobin_a1c_labeled.query('HASDIABETES == 1')['VALUE'].astype('float'),
    hemoglobin_a1c_labeled.query('HASDIABETES == 0')['VALUE'].astype('float'), 
    method="auto"
)
print(f"U1: {U1}, p-value: {p}")

In [ ]:
statistic, p_value = sp.stats.ks_2samp(
    hemoglobin_a1c_labeled.query('HASDIABETES == 1')['VALUE'].astype('float'),
    hemoglobin_a1c_labeled.query('HASDIABETES == 0')['VALUE'].astype('float')
)
print(f"KS Statistic: {statistic}, p-value: {p_value}")

# Discussion Points
1. Why is it that we observe a (roughly) bi-modal distribution in A1C values among patients without a Type 2 Diabetes diagnosis?
2. Why do we observe right skewness and high A1C outliers in diabetic patients, while observing a median A1C value and prominent mode that is well within the defined normal range?
3. How can we extend this analysis to add additional nuance (additional lab tests, adding data from other tables, partitioning patients further based on \
   other diagnoses, medications, demographics etc)?

# II. Open Ended Exploration & Extensions to Initial Analysis
Try to see if you can extend the initial analysis that we did on Hemoglobin A1C and diabetes

1. Are there any other labs that you could pull data for and compare distributions for between Type-2 and non-Type2 patients
2. Could you further subdivide the Type 2 and non-Type 2 patients into additional subsets based on other diagnoses/demographics/medication status?

**Click Below for Hints**

<details>
    <summary>Hint 1</summary>
    <p>The medications table has both a <code>'REASONCODE'</code> abd <code>'REASONDESCRIPTION'</code> column which describe why the medication was prescribed</p>
</details>
<details>
    <summary>Hint 2</summary>
    <p>If you aren't sure which LOINC to query for a particular lab for, you can always use the <code>Series.str.contains('_PATTERN_', case=False)</code> technique to perform a string search</p>
</details>
<details>
    <summary>Hint 3</summary>
    <p>If you want to search for the SNOMED CT code for other diagnoses, such as Pre-diabetes, you can do that via <a href="https://browser.ihtsdotools.org/?perspective=full&conceptId1=9414007&edition=MAIN/2025-06-01&release=&languages=en">SNOMED Browser</a></p>
    <p> You can also take advantage of string/keyword search, again via <code>Series.str.contains('_PATTERN_', case=False)</code></p>
</details>